Install and import packages

In [ ]:
!pip install git+https://github.com/LLaVA-VL/LLaVA-NeXT.git
!pip install halo
!pip install flash-attn
!pip install av
!pip install --upgrade accelerate bitsandbytes
!pip install git+https://github.com/huggingface/transformers.git
!pip install git+https://github.com/shumingma/transformers.git

  Cloning https://github.com/LLaVA-VL/LLaVA-NeXT.git to /tmp/pip-req-build-2uo3zkth
  Running command git clone --filter=blob:none --quiet https://github.com/LLaVA-VL/LLaVA-NeXT.git /tmp/pip-req-build-2uo3zkth
  Resolved https://github.com/LLaVA-VL/LLaVA-NeXT.git to commit 09e5840d5589ad2d6a8656c0a60f21ae134b3309
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llava: filename=llava-1.7.0.dev0-py3-none-any.whl size=334056 sha256=80ad7fbdd13505db4df562ab6ce774050e424cb78b37e902eb2c251fc363cb91
  Stored in directory: /tmp/pip-ephem-wheel-cache-kybfnmyr/wheels/c0/c8/54/be0fea0529441b125bf656360b14e7f7458b1ee3d4c6f329fb
Successfully built llava
  Preparing metadata (setup.py) ... done
  Created wheel for halo: filename=halo-0.0.31-py3-none-any.whl size=11232 sha256=a3f0f39e5ea8fb48da2570ce4db11b6371909d5d26738ae77b143688d2664be7
  Stored in directory: /root/.cache/pip/wheels/2e/b1/0a/6

Imports and setup the model

In [ ]:
# Imports... fucken obviously
from transformers import BitsAndBytesConfig, LlavaNextVideoForConditionalGeneration, LlavaNextVideoProcessor
import torch
import av
import numpy as np
from huggingface_hub import hf_hub_download
import os
from PIL import Image
import urllib.request
import tempfile
from urllib.parse import urlparse
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from google.colab import drive
drive.mount('/gdrive')


ValueError: mount failed

In [ ]:
# Setup and load model
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

processor = LlavaNextVideoProcessor.from_pretrained("llava-hf/LLaVA-NeXT-Video-7B-hf")
model = LlavaNextVideoForConditionalGeneration.from_pretrained(
    "llava-hf/LLaVA-NeXT-Video-7B-hf",
    quantization_config=quantization_config,
    device_map='auto'
)

# model_id = "microsoft/bitnet-b1.58-2B-4T"

# # Load tokenizer and model
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     torch_dtype=torch.bfloat16
# )

In [ ]:
# Prompt setup
conversation = [
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": """USER:
You must output exactly and only in the following strict format:

DESCRIPTION: <frame-by-frame, extremely detailed and verbose play-by-play of the video. In your description, answer WHO (gender, age, physical motions), WHAT (what actions are happening, how are the objects/people interacting), WHEN (dates, times of day if visible), WHERE (country if possible, objects in frame, buildings), and HOW (what tools are being used, how people are physically moving). Be extremely detailed and verbose, frame-by-frame, not general.>

MATURE: true or false (Is the content adult or mature?)
CHILD: true or false (Does the content show children?)

Rules:
- Follow with 'Description:' — a very verbose frame-by-frame play-by-play including WHO, WHAT, WHEN, WHERE, HOW.
- Then, 'MATURE:' (true/false).
- Then, 'CHILD:' (true/false).
- No assistant commentary, no extra talking. Only output the four fields above.
- If you don't know a fact, state "Unknown" inside the DESCRIPTION — don't invent.

Important:
- If you fail to follow the output format exactly, you will be penalized.
- Do not invent information you cannot clearly see in the video.

Begin""",
            },
            {
                "type": "video",
            },
        ],
    }
]


# conversation_2 = [
#       {
#           "role": "user",
#           "content": [
#               {"type": "text", "text": "What do you see in this video?"},
#               {"type": "video"},
#               ],
#       },
# ]

prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)
# prompt_2 = processor.apply_chat_template(conversation_2, add_generation_prompt=True)


In [ ]:
from math import e
def get_total_frames(container):
    stream = container.streams.video[0]
    if stream.frames:
        return stream.frames
    # fallback: count frames manually
    return sum(1 for _ in container.decode(video=0))

# container = av.open(video_path_2)

# # sample uniformly 8 frames from the video (we can sample more for longer videos)
# total_frames = container.streams.video[0].frames
# indices = np.arange(0, total_frames, total_frames / 8).astype(int)
# clip_karate = read_video_pyav(container, indices)
def inferenceModel(file_path):
    # Read 8 frames (or single image) using read_media
    clip = read_media(file_path, num_frames=50)  # handles both video & image

    # Process and generate text
    inputs = processor([prompt], videos=[clip], padding=True, return_tensors="pt").to(model.device)
    generate_kwargs = {"max_new_tokens": 200, "do_sample": True, "top_p": 0.9}

    output = model.generate(**inputs, **generate_kwargs)
    generated_text = processor.batch_decode(output, skip_special_tokens=True)[0]

    return generated_text

def read_media(input_obj, indices=None, num_frames=8):
    '''
    Accepts either a file path (image or video) or a PyAV container.

    Args:
        input_obj (str or av.container.input.InputContainer): Path or container.
        indices (List[int], optional): Frame indices to extract.
        num_frames (int): Frames to sample if indices not provided.

    Returns:
        np.ndarray: Frames array (N, H, W, 3).
    '''
    if isinstance(input_obj, str):
        ext = os.path.splitext(input_obj)[-1].lower()

        if ext in ['.jpg', '.jpeg', '.png']:
            img = Image.open(input_obj).convert("RGB")
            return np.stack([np.array(img)])

        # Assume it's a video file
        container = av.open(input_obj)
        return _read_from_container(container, indices, num_frames)

    elif isinstance(input_obj, av.container.input.InputContainer):
        return _read_from_container(input_obj, indices, num_frames)

    else:
        raise TypeError("Input must be a file path or PyAV container.")

def _read_from_container(container, indices=None, num_frames=8):
    '''
    Internal function to read frames from a PyAV container.

    Args:
        container (av.container.input.InputContainer): Opened video container.
        indices (List[int], optional): Specific frame indices to extract.
        num_frames (int): If indices not provided, sample this many frames.

    Returns:
        np.ndarray: Frames array (N, H, W, 3).
    '''
    stream = container.streams.video[0]

    try:
        total_frames = stream.frames
    except AttributeError:
        total_frames = None

    if not total_frames or total_frames <= 0:
        # Fallback to counting frames
        total_frames = sum(1 for _ in container.decode(video=0))
        container.seek(0)

    if indices is None:
      num_frames = max(1, int(total_frames * 0.3))  # 30% of total frames
    indices = np.linspace(0, total_frames - 1, num_frames).astype(int)

    frames = []
    for i, frame in enumerate(container.decode(video=0)):
        if i > indices[-1]:
            break
        if i in indices:
            frames.append(frame.to_ndarray(format="rgb24"))

    if not frames:
        raise ValueError("No frames extracted from video.")

    return np.stack(frames)


# Download video from the hub
output = ''
urls = [
    "https://www.asiaterra.info/videos/%d0%a2%d0%b0%d0%bb%d0%b8%d0%b1%d1%8b.mp4",
    "https://www.asiaterra.info/videos/01032023.mp4",
    "https://www.asiaterra.info/videos/01052023.mp4",
    "https://www.asiaterra.info/videos/01122024.mp4",
    "https://www.asiaterra.info/videos/02112023.mp4",
    "https://www.asiaterra.info/videos/02272024.mp4",
    "https://www.asiaterra.info/videos/03062024.mp4",
    "https://www.asiaterra.info/videos/05102024.mp4",
    "https://www.asiaterra.info/videos/051720222.mp4",
    "https://www.asiaterra.info/videos/05252023.mp4",
    "https://www.asiaterra.info/videos/05272022.mp4",
    "https://www.asiaterra.info/videos/05282022.mp4",
    "https://www.asiaterra.info/videos/06012023.mp4",
    "https://www.asiaterra.info/videos/06132023_1.mp4",
    "https://www.asiaterra.info/videos/06132023_2.mp4",
    "https://www.asiaterra.info/videos/07.11.2021.mp4",
    "https://www.asiaterra.info/videos/07032022_1.mp4",
    "https://www.asiaterra.info/videos/07032022_2.mp4",
    "https://www.asiaterra.info/videos/07032022_3.mp4",
    "https://www.asiaterra.info/videos/07032022_4.mp4",
    "https://www.asiaterra.info/videos/07032022_5.mp4",
    "https://www.asiaterra.info/videos/07032022_6.mp4",
    "https://www.asiaterra.info/videos/07252022.mp4",
    "https://www.asiaterra.info/videos/07252022_2.mp4",
    "https://www.asiaterra.info/videos/07252022_4.mp4",
    "https://www.asiaterra.info/videos/090420221.mp4",
    "https://www.asiaterra.info/videos/090420222.mp4",
    "https://www.asiaterra.info/videos/09092023_1.mp4",
    "https://www.asiaterra.info/videos/09092023_2.mp4",
    "https://www.asiaterra.info/videos/09162022_1.mp4",
    "https://www.asiaterra.info/videos/09162022_2.mp4",
    "https://www.asiaterra.info/videos/4%20Funny%20Anonymous.mp4",
    "https://www.asiaterra.info/videos/2022-02-02%2012-01-26.mp4",
    "https://www.asiaterra.info/videos/11142023.mp4",
    "https://www.asiaterra.info/videos/14062024.mp4",
    "https://www.asiaterra.info/videos/14092024_1.mp4",
    "https://www.asiaterra.info/videos/14092024_2.mp4",
    "https://www.asiaterra.info/videos/14092024_3.mp4",
    "https://www.asiaterra.info/videos/14092024_4.mp4",
    "https://www.asiaterra.info/videos/14122024.mp4",
    "https://www.asiaterra.info/videos/16052025.mp4",
    "https://www.asiaterra.info/videos/16052025_2.mp4",
    "https://www.asiaterra.info/videos/20052022_2.mp4",
    "https://www.asiaterra.info/videos/23072022.mp4",
    "https://www.asiaterra.info/videos/281548172_849954319731518_5793844223177448526_n.mp4",
    "https://www.asiaterra.info/videos/1658595328.mp4",
    "https://www.asiaterra.info/videos/Bedana.mp4",
    "https://www.asiaterra.info/videos/Madina_Ishmurzina.mp4",
    "https://www.asiaterra.info/videos/Nurzhan_is_pavlodara.mp4",
    "https://www.asiaterra.info/videos/Ozodlik_radiosi_Karimov_pro_geyeev.mp4",
    "https://www.asiaterra.info/videos/TjBFpmX-3H7JaX.mp4",
    "https://www.asiaterra.info/videos/Uzbek_zapal.mp4",
    "https://www.asiaterra.info/videos/Vsya_pravda_o_Bozumayeve.mp4",
    "https://www.asiaterra.info/videos/Zhertva_iznasilovanya.mp4",
    "https://www.asiaterra.info/videos/asia.mp4",
    "https://www.asiaterra.info/videos/obrez.mp4",
    "https://www.papytane.com/mp4/accrobra.mp4",
    "https://www.papytane.com/mp4/airelles.mp4",
    "https://www.papytane.com/mp4/anni18an.mp4",
    "https://www.papytane.com/mp4/arnaudin.mp4",
    "https://www.papytane.com/mp4/aulacdes.mp4",
    "https://www.papytane.com/mp4/baladeda.mp4",
    "https://www.papytane.com/mp4/barbizon.mp4",
    "https://www.papytane.com/mp4/belleima.mp4",
    "https://www.papytane.com/mp4/boiteame.mp4",
    "https://www.papytane.com/mp4/bonjour.mp4",
    "https://www.papytane.com/mp4/camimenu.mp4",
    "https://www.papytane.com/mp4/camiocel.mp4",
    "https://www.papytane.com/mp4/canalett.mp4",
    "https://www.papytane.com/mp4/caranca.mp4",
    "https://www.papytane.com/mp4/casteil.mp4",
    "https://www.papytane.com/mp4/celuiqui.mp4",
    "https://www.papytane.com/mp4/cerbere.mp4",
    "https://www.papytane.com/mp4/chalam.mp4",
    "https://www.papytane.com/mp4/chamaran.mp4",
    "https://www.papytane.com/mp4/chamboux.mp4",
    "https://www.papytane.com/mp4/charjema.mp4",
    "https://www.papytane.com/mp4/chevress.mp4",
    "https://www.papytane.com/mp4/cousines.mp4",
    "https://www.papytane.com/mp4/cygne.mp4",
    "https://www.papytane.com/mp4/dejeuner.mp4",
    "https://www.papytane.com/mp4/descente.mp4",
    "https://www.papytane.com/mp4/deslupin.mp4",
    "https://www.papytane.com/mp4/devoirs.mp4",
    "https://www.papytane.com/mp4/doucefra.mp4",
    "https://www.papytane.com/mp4/dugivre.mp4",
    "https://www.papytane.com/mp4/dunlespl.mp4",
    "https://www.papytane.com/mp4/ecureuil.mp4",
    "https://www.papytane.com/mp4/edelweis.mp4",
    "https://www.papytane.com/mp4/evolrond.mp4",
    "https://www.papytane.com/mp4/faneurs.mp4",
    "https://www.papytane.com/mp4/fauvisme.mp4",
    "https://www.papytane.com/mp4/fedr2004.mp4",
    "https://www.papytane.com/mp4/galapago.mp4",
    "https://www.papytane.com/mp4/glisse.mp4",
    "https://www.papytane.com/mp4/gouloux.mp4",
    "https://www.papytane.com/mp4/griselle.mp4",
    "https://www.papytane.com/mp4/guillera.mp4",
    "https://www.papytane.com/mp4/herta.mp4",
    "https://www.papytane.com/mp4/holzarte.mp4",
    "https://www.papytane.com/mp4/jai18ans.mp4",
    "https://www.papytane.com/mp4/jaimarch.mp4",
    "https://www.papytane.com/mp4/jenaurai.mp4",
    "https://www.papytane.com/mp4/kakuetta.mp4",
    "https://www.papytane.com/mp4/kari14an.mp4",
    "https://www.papytane.com/mp4/kari17an.mp4",
    "https://www.papytane.com/mp4/kiri.mp4",
    "https://www.papytane.com/mp4/laboucle.mp4",
    "https://www.papytane.com/mp4/labyvert.mp4",
    "https://www.papytane.com/mp4/lacblanc.mp4",
    "https://www.papytane.com/mp4/lacdulou.mp4",
    "https://www.papytane.com/mp4/lacechau.mp4",
    "https://www.papytane.com/mp4/lacforle.mp4",
    "https://www.papytane.com/mp4/lacvert.mp4",
    "https://www.papytane.com/mp4/lagaure.mp4",
    "https://www.papytane.com/mp4/lamaison.mp4",
    "https://www.papytane.com/mp4/lelongde.mp4",
    "https://www.papytane.com/mp4/lena18an.mp4",
    "https://www.papytane.com/mp4/lenabrev.mp4",
    "https://www.papytane.com/mp4/lenanage.mp4",
    "https://www.papytane.com/mp4/lepontde.mp4",
    "https://www.papytane.com/mp4/les12jou.mp4",
    "https://www.papytane.com/mp4/lesavent.mp4",
    "https://www.papytane.com/mp4/leshaute.mp4",
    "https://www.papytane.com/mp4/lessapin.mp4",
    "https://www.papytane.com/mp4/lessolda.mp4",
    "https://www.papytane.com/mp4/lestrois.mp4",
    "https://www.papytane.com/mp4/lesucdu.mp4",
    "https://www.papytane.com/mp4/lispach.mp4",
    "https://www.papytane.com/mp4/louisfer.mp4",
    "https://www.papytane.com/mp4/lucabrev.mp4",
    "https://www.papytane.com/mp4/lucanage.mp4",
    "https://www.papytane.com/mp4/marais.mp4",
    "https://www.papytane.com/mp4/marcevol.mp4",
    "https://www.papytane.com/mp4/marquez2.mp4",
    "https://www.papytane.com/mp4/marqueze.mp4",
    "https://www.papytane.com/mp4/matemale.mp4",
    "https://www.papytane.com/mp4/mauperth.mp4",
    "https://www.papytane.com/mp4/merciaux.mp4",
    "https://www.papytane.com/mp4/merevill.mp4",
    "https://www.papytane.com/mp4/mesquer.mp4",
    "https://www.papytane.com/mp4/mexico.mp4",
    "https://www.papytane.com/mp4/michrene.mp4",
    "https://www.papytane.com/mp4/montgolf.mp4",
    "https://www.papytane.com/mp4/mourcaou.mp4",
    "https://www.papytane.com/mp4/neigepis.mp4",
    "https://www.papytane.com/mp4/nightbef.mp4",
    "https://www.papytane.com/mp4/noel2010.mp4",
    "https://www.papytane.com/mp4/noel2011.mp4",
    "https://www.papytane.com/mp4/noel2012.mp4",
    "https://www.papytane.com/mp4/noel2015.mp4",
    "https://www.papytane.com/mp4/noel2016.mp4",
    "https://www.papytane.com/mp4/nogentel.mp4",
    "https://www.papytane.com/mp4/nogentra.mp4",
    "https://www.papytane.com/mp4/oniria.mp4",
    "https://www.papytane.com/mp4/opouperi.mp4",
    "https://www.papytane.com/mp4/ozoirien.mp4",
    "https://www.papytane.com/mp4/paris.mp4",
    "https://www.papytane.com/mp4/parproca.mp4",
    "https://www.papytane.com/mp4/passea10.mp4",
    "https://www.papytane.com/mp4/passerel.mp4",
    "https://www.papytane.com/mp4/patinoir.mp4",
    "https://www.papytane.com/mp4/paysborn.mp4",
    "https://www.papytane.com/mp4/peintres.mp4",
    "https://www.papytane.com/mp4/penestin.mp4",
    "https://www.papytane.com/mp4/pennylan.mp4",
    "https://www.papytane.com/mp4/petitcan.mp4",
    "https://www.papytane.com/mp4/plustard.mp4",
    "https://www.papytane.com/mp4/pontcarr.mp4",
    "https://www.papytane.com/mp4/promegli.mp4",
    "https://www.papytane.com/mp4/promenad.mp4",
    "https://www.papytane.com/mp4/promeozo.mp4",
    "https://www.papytane.com/mp4/randonav.mp4",
    "https://www.papytane.com/mp4/randopis.mp4",
    "https://www.papytane.com/mp4/rentreed.mp4",
    "https://www.papytane.com/mp4/resteala.mp4",
    "https://www.papytane.com/mp4/retjons.mp4",
    "https://www.papytane.com/mp4/retourve.mp4",
    "https://www.papytane.com/mp4/revedeno.mp4",
    "https://www.papytane.com/mp4/saintagn.mp4",
    "https://www.papytane.com/mp4/saintbri.mp4",
    "https://www.papytane.com/mp4/saintere.mp4",
    "https://www.papytane.com/mp4/salettes.mp4",
    "https://www.papytane.com/mp4/sapins.mp4",
    "https://www.papytane.com/mp4/sardane.mp4",
    "https://www.papytane.com/mp4/senteyre.mp4",
    "https://www.papytane.com/mp4/serigoul.mp4",
    "https://www.papytane.com/mp4/silentni.mp4",
    "https://www.papytane.com/mp4/solferin.mp4",
    "https://www.papytane.com/mp4/solsora.mp4",
    "https://www.papytane.com/mp4/starwars.mp4",
    "https://www.papytane.com/mp4/stfront.mp4",
    "https://www.papytane.com/mp4/surlespa.mp4",
    "https://www.papytane.com/mp4/surnette.mp4",
    "https://www.papytane.com/mp4/suscinio.mp4",
    "https://www.papytane.com/mp4/thefroze.mp4",
    "https://www.papytane.com/mp4/tombelan.mp4",
    "https://www.papytane.com/mp4/toupisso.mp4",
    "https://www.papytane.com/mp4/tourbier.mp4",
    "https://www.papytane.com/mp4/unmatina.mp4",
    "https://www.papytane.com/mp4/untraine.mp4",
    "https://www.papytane.com/mp4/vacances.mp4",
    "https://www.papytane.com/mp4/varennes.mp4",
    "https://www.papytane.com/mp4/vauxlevi.mp4",
    "https://www.papytane.com/mp4/venise.mp4",
    "https://www.papytane.com/mp4/vernet.mp4",
    "https://www.papytane.com/mp4/vttbisca.mp4",
    "https://www.papytane.com/mp4/yaleprin.mp4"
]

print("Start")

urls_len = len(urls)
url_count = 0
for url in urls:
  url_count += 1
  print(f"{url_count}/{urls_len}")
  try:
    generated_text = inferenceModel(url)
    # print("------------------------------------------------------------")
    # print(url)
    description_only = generated_text.replace("""USER:
You must output exactly and only in the following strict format:

DESCRIPTION: <frame-by-frame, extremely detailed and verbose play-by-play of the video. In your description, answer WHO (gender, age, physical motions), WHAT (what actions are happening, how are the objects/people interacting), WHEN (dates, times of day if visible), WHERE (country if possible, objects in frame, buildings), and HOW (what tools are being used, how people are physically moving). Be extremely detailed and verbose, frame-by-frame, not general.>

MATURE: true or false (Is the content adult or mature?)
CHILD: true or false (Does the content show children?)

Rules:
- Follow with 'Description:' — a very verbose frame-by-frame play-by-play including WHO, WHAT, WHEN, WHERE, HOW.
- Then, 'MATURE:' (true/false).
- Then, 'CHILD:' (true/false).
- No assistant commentary, no extra talking. Only output the four fields above.
- If you don't know a fact, state "Unknown" inside the DESCRIPTION — don't invent.

Important:
- If you fail to follow the output format exactly, you will be penalized.
- Do not invent information you cannot clearly see in the video.

Begin""", "")
    # print(description_only)
    # print("------------------------------------------------------------")
    with open('/gdrive/My Drive/Serca/descriptions_new.txt', 'a') as f:
      f.write("\n")
      f.write(url + "\n")
      f.write(description_only + "\n")
  except Exception as e:
    print(f"Error processing {url}: {e}")
    continue

print("\n")
print("Done")


Start
1/208
Error processing https://www.asiaterra.info/videos/%d0%a2%d0%b0%d0%bb%d0%b8%d0%b1%d1%8b.mp4: name 'os' is not defined
2/208
Error processing https://www.asiaterra.info/videos/01032023.mp4: name 'os' is not defined
3/208
Error processing https://www.asiaterra.info/videos/01052023.mp4: name 'os' is not defined
4/208
Error processing https://www.asiaterra.info/videos/01122024.mp4: name 'os' is not defined
5/208
Error processing https://www.asiaterra.info/videos/02112023.mp4: name 'os' is not defined
6/208
Error processing https://www.asiaterra.info/videos/02272024.mp4: name 'os' is not defined
7/208
Error processing https://www.asiaterra.info/videos/03062024.mp4: name 'os' is not defined
8/208
Error processing https://www.asiaterra.info/videos/05102024.mp4: name 'os' is not defined
9/208
Error processing https://www.asiaterra.info/videos/051720222.mp4: name 'os' is not defined
10/208
Error processing https://www.asiaterra.info/videos/05252023.mp4: name 'os' is not defined
11/2

# This is the post processing code


In [ ]:
import re
from google.colab import drive
drive.mount('/gdrive')

def clean_description(text):
    # Normalize whitespace
    text = re.sub(r'[\n\r\t]+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    # Keep only alphanumeric characters, apostrophes, and spaces
    text = re.sub(r"[^a-zA-Z0-9' ]+", '', text)
    # Escape apostrophes for SQL
    return text.strip().replace("'", "''")

def parse_descriptions(filepath):
    entries = []
    with open(filepath, 'r', encoding='utf-8') as file:
        content = file.read()

    # Find all video URLs (mp4, gif, webm)
    url_pattern = r'https?://[^\s]+?\.(?:mp4|gif|webm)'
    urls = re.findall(url_pattern, content)
    blocks = re.split(url_pattern, content)

    for i, block in enumerate(blocks[1:]):  # skip leading empty match
        url = urls[i]

        # Capture description block
        description_match = re.search(r'Description:([\s\S]*?)(?=MATURE:|CHILD:|https://|$)', block, re.IGNORECASE)
        raw_description = description_match.group(1).strip() if description_match else "Unknown"
        cleaned_description = clean_description(raw_description)

        # Capture MATURE and CHILD flags
        mature_match = re.search(r'MATURE:\s*(true|false)', block, re.IGNORECASE)
        child_match = re.search(r'CHILD:\s*(true|false)', block, re.IGNORECASE)

        mature = mature_match.group(1).lower() == 'true' if mature_match else False
        child = child_match.group(1).lower() == 'true' if child_match else False

        entries.append((url, cleaned_description, mature, child, 'green'))

    return entries

def generate_sql(entries, table_name="urls"):
    sql_lines = [f"INSERT INTO {table_name} (url, description, mature, child, flag) VALUES"]
    for url, desc, mature, child, flag in entries:
        line = f"('{url}', '{desc}', {str(mature).lower()}, {str(child).lower()}, '{flag}')"
        sql_lines.append(line)
    return ",\n".join(sql_lines) + ";"

# Run the script
if __name__ == "__main__":
    input_file = '/gdrive/My Drive/Serca/descriptions_new.txt'
    output_file = '/gdrive/My Drive/Serca/query.sql'

    entries = parse_descriptions(input_file)
    sql = generate_sql(entries)
    print(sql)
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(sql)

    print(f"Generated SQL saved to {output_file}")


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
INSERT INTO urls (url, description, mature, child, flag) VALUES,
('https://www.asiaterra.info/videos/%d0%a2%d0%b0%d0%bb%d0%b8%d0%b1%d1%8b.mp4', 'framebyframe extremely detailed and verbose playbyplay of the video In your description answer WHO gender age physical motions WHAT what actions are happening how are the objectspeople interacting WHEN dates times of day if visible WHERE country if possible objects in frame buildings and HOW what tools are being used how people are physically moving Be extremely detailed and verbose framebyframe not general', true, true, 'green'),
('https://www.asiaterra.info/videos/01032023.mp4', 'framebyframe extremely detailed and verbose playbyplay of the video In your description answer WHO gender age physical motions WHAT what actions are happening how are the objectspeople interacting WHEN dates times of day if visible WHERE country if poss